# CSS selector
## 원티드
- recruit_position : `.JobHeader_className__HttDA h2`
- is_hiring : `button.ApplyBtn_button__BT821`
- recruit_thumb : `.JobImage_JobImage__slide__w4mSW img`
- recruit_url 

## 사람인
- recruit_position : `.tit_job`(문장형)
- is_hiring : `.wrap_jv_header .btn_apply a` 첫번째거.
	- 홈페이지 지원 : `.sri_btn_homepage_apply`
	- 일반 지원 : `.sri_btn_immediately`
	- 마감됨 : `.sri_btn_expired_apply`
- recruit_thumb : X
- recruit_url 

## 잡코리아
- recruit_position : `div.recruitment > table td.header`
- is_hiring : `div.sumBtn button` "title" attribute
- recruit_thumb : X
- recruit_url 

In [14]:
import requests as req
from bs4 import BeautifulSoup as bs

In [15]:
keyword = '무신사'

V_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
SAVE_PATH = './datas/recruit'

## 사람인

In [66]:
# 사람인은 파라미터가 좀 많다.
SARAMIN_HOME = 'https://www.saramin.co.kr'
SARAMIN_SEARCH = SARAMIN_HOME + '/zf_user/search?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword={}'

soup = bs(req.get(SARAMIN_SEARCH.format(keyword, 1), headers={'user-agent': V_AGENT}).text, 'lxml')

# 회사명 검사
urls = []
first_corp_name = soup.select_one('.corp_name b').text

cont_selector = '#recruit_info_list > .content > div'
corp_name_selector = cont_selector + ':nth-child({}) > .area_corp b'
atag_selector = cont_selector + ':nth-child({}) h2.job_tit a'

## 잡코리아

In [78]:
JOBKR_HOME = 'https://www.jobkorea.co.kr'
JOBKR_SEARCH = JOBKR_HOME + '/Search/?stext={}'

soup = bs(req.get(JOBKR_SEARCH.format(keyword, 1), headers={'user-agent': V_AGENT}).text, 'lxml')

# 회사명 검사
urls = []
first_corp_name = soup.select_one('.post-list-corp strong').text

cont_selector = '#content div.cnt-list-wrap .recruit-info > .lists .list-default li'
corp_name_selector = cont_selector + ':nth-child({}) .post-list-corp strong'
atag_selector = cont_selector + ':nth-child({}) .post-list-info a'

## 사람인 & 잡코리아 공통 로직

In [81]:
# 검색 결과가 있는 경우에만 공고 링크 따기
if first_corp_name is not None:
    
    # 있는 채용공고 만큼 돌린다.
    for i in range(len(soup.select(cont_selector))):
        corp_name_selector = corp_name_selector.format((i+1), 1)
        atag_selector = atag_selector.format((i+1), 1)
        
        # 만료한 경우
        if soup.select_one(corp_name_selector) is None:
            break
        
        # 최초 검색된 기업명과 검색 기업명이 상이해진 경우
        elif first_corp_name == soup.select_one(corp_name_selector).text:
            urls.append(JOBKR_HOME + soup.select_one(atag_selector)['href'])
urls

['https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/R

## 원티드

In [25]:
import requests as req
import pandas as pd

import json
V_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

def get_recruit_details(uid):
    WANTED_JOB = 'https://www.wanted.co.kr/api/v4/jobs/{}'
    
    # 크롤링 시작
    headers = {
        'user-agent': V_AGENT,
        'Referer': f'https://www.wanted.co.kr/wd/{uid}'
    }
    result = json.loads(req.get(WANTED_JOB.format(uid, 1), headers=headers).text)
    result = result['job']['detail']['intro'][:200]
    
    return result

def get_recruit_info(keyword):
    WANTED_SEARCH ='https://www.wanted.co.kr/api/chaos/search/v1/results?1691048648606=&query={}&country=kr&job_sort=company.response_rate_order&years=-1&locations=all'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': f'https://www.wanted.co.kr/search?query={keyword.encode("UTF-8").decode("iso-8859-1")}'
    }
    result = json.loads(req.get(WANTED_SEARCH.format(keyword, 1), headers=headers).text)

    result = result['positions']['data']
    
    return pd.DataFrame({
        'recruit_uid': [{el['id']} for el in result],
        'recruit_url': [f"https://www.wanted.co.kr/wd/{el['id']}" for el in result],
        'recruit_position': [el['position'] for el in result],
        'recruit_thumb': [el['title_img']['origin'] for el in result],
        'recruit_desc': [get_recruit_details(el['id']) for el in result]
    })


In [26]:
get_recruit_info('카카오')

,recruit_uid,recruit_url,recruit_position,recruit_thumb,recruit_desc
0,{176320},https://www.wanted.co.kr/wd/176320,[신입] iOS개발,https://static.wanted.co.kr/images/company/351...,"카카오헬스케어는 ‘헬스케어를 활용하는 모든 사람을 위한 동반자, 친구, 비서’라는 ..."
1,{172569},https://www.wanted.co.kr/wd/172569,iOS개발자 (3년 이상),https://static.wanted.co.kr/images/company/351...,"카카오헬스케어는 ‘헬스케어를 활용하는 모든 사람을 위한 동반자, 친구, 비서’라는 ..."
2,{175906},https://www.wanted.co.kr/wd/175906,국내/해외 주식 업무기획 PM,https://static.wanted.co.kr/images/company/209...,"카카오페이증권은 어렵고 복잡한 투자의 문턱을 낮추고,\n생활 속 가장 가까운 금융투..."
3,{174812},https://www.wanted.co.kr/wd/174812,헬스케어 모바일 서비스 기획자 (1년 계약직),https://static.wanted.co.kr/images/company/351...,"카카오헬스케어는 ‘헬스케어를 활용하는 모든 사람을 위한 동반자, 친구, 비서’라는 ..."
4,{173529},https://www.wanted.co.kr/wd/173529,스크린앱개발팀 앱 개발 및 관리 팀장,https://static.wanted.co.kr/images/company/368...,"카카오VX는 대한민국 스크린골프 대표 브랜드인 \n‘티업비전2’, ‘티업비전, ‘지..."
5,{156751},https://www.wanted.co.kr/wd/156751,Biology AI Researcher,https://static.wanted.co.kr/images/company/225...,팀소개\n\n카카오브레인은 AI 기술로 새로운 서비스를 발굴하고 기존의 문제를 재해...
6,{149197},https://www.wanted.co.kr/wd/149197,Inference Engine Engineer,https://static.wanted.co.kr/images/company/225...,팀소개\n\n카카오브레인은 AI 기술로 새로운 서비스를 발굴하고 기존의 문제를 재해...
7,{151828},https://www.wanted.co.kr/wd/151828,Applied AI/ML Engineer,https://static.wanted.co.kr/images/company/225...,팀소개\n\n카카오브레인은 AI 기술로 새로운 서비스를 발굴하고 기존의 문제를 재해...
8,{159350},https://www.wanted.co.kr/wd/159350,Backend Engineer(AI Image Service),https://static.wanted.co.kr/images/company/225...,팀소개\n\n카카오브레인은 AI 기술로 새로운 서비스를 발굴하고 기존의 문제를 재해...
9,{174324},https://www.wanted.co.kr/wd/174324,Business Operations 담당 (Infra & Platform),https://static.wanted.co.kr/images/company/225...,"팀소개\n\n- 카카오브레인은 생각하지 못한 물음으로부터 기술을 발전시키고, 모두의..."
